In [1]:
from bs4 import BeautifulSoup
import requests
import re

# webscrapes highest profit margin available in a given state
def get_pmargin(state):
    url = "https://www.oddsshopper.com/tools/arbitrage/{}".format(state)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    try:
        pmargin = soup.find('p', class_="MuiTypography-root jss90 MuiTypography-body1").text[1:]     
    except:
        pmargin = None
    return pmargin, url

In [4]:
import smtplib

# notifies through email: subject contains profit margin and email text contains link to the opportunity
def notify(pmargin, url):
    
    # user
    gmail_user = <your gmail address without @gmail.com>
    gmail_password = <your gmail password>
    
    # email properties
    sent_from = gmail_user
    subject = "Arbitrage opportunity found: {}".format(pmargin)
    to = ["joeomatoi2023@u.northwestern.edu"]
    email_text = """Access opportunity here: {}""".format(url)
    message = 'Subject: {}\n\n{}'.format(subject, email_text)
    
    #email send request
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, message)
        server.close()

        print ('Email sent!')
    except Exception as e:
        print(e)
        print ('Something went wrong...')

In [5]:
# runs the notify() function only when there is a new profit margin of over 2% available
def monitor_and_send(state):
    global pmargin
    
    try:
        oldpmargin = pmargin
    except:
        oldpmargin = "DNE"
        
    pmargin = get_pmargin(state)[0]
    url = get_pmargin(state)[1]
    
    if pmargin == None:
        return
        
    elif oldpmargin == pmargin:
        return
    
    elif float(pmargin[:-1]) <= 0.2:
        return
        
    else:
        notify(pmargin, url)  
        return

In [ ]:
import schedule

# run the function every 5 hours
def run():
    monitor_and_send("in")
      
schedule.every(5).hours.do(run)
 
while True:
    schedule.run_pending()